In [21]:
!nvidia-smi

Sun Jul 16 03:03:13 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   78C    P0    32W /  70W |   2603MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [22]:
!git clone https://github.com/HusseinMansourMohd/Enhancing-MedViT-Incorporating-Adapter-Modules-for-Improved-Medical-Image-Classification


Cloning into 'Enhancing-MedViT-Incorporating-Adapter-Modules-for-Improved-Medical-Image-Classification'...
remote: Enumerating objects: 134, done.
remote: Counting objects: 100% (134/134), done.
remote: Compressing objects: 100% (102/102), done.
remote: Total 134 (delta 64), reused 84 (delta 30), pack-reused 0
Receiving objects: 100% (134/134), 43.69 KiB | 1.82 MiB/s, done.
Resolving deltas: 100% (64/64), done.


In [23]:
import os
print(os.listdir())

['MedVit_adapter.ipynb', 'MedVit_adapter.py', 'utils.py', '.git', 'Enhancing-MedViT-Incorporating-Adapter-Modules-for-Improved-Medical-Image-Classification', '__pycache__', 'README.md']


In [24]:
import sys
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision.utils
import torchvision.datasets as dsets
from torchsummary import summary


In [25]:
!pip install timm
!pip install einops

In [26]:
cd /content/Enhancing-MedViT-Incorporating-Adapter-Modules-for-Improved-Medical-Image-Classification/

/content/Enhancing-MedViT-Incorporating-Adapter-Modules-for-Improved-Medical-Image-Classification


In [27]:
#print(os.listdir())
import MedVit_adapter
print(dir(MedVit_adapter))

['Adapter', 'ConvBNReLU', 'DropPath', 'ECALayer', 'ECB', 'E_MHSA', 'F', 'LTB', 'LocalityFeedForward', 'MHCA', 'MedViT', 'MedViT_base', 'MedViT_large', 'MedViT_small', 'Mlp', 'NORM_EPS', 'PatchEmbed', 'SELayer', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '_make_divisible', 'checkpoint', 'h_sigmoid', 'h_swish', 'math', 'merge_pre_bn', 'nn', 'partial', 'rearrange', 'register_model', 'torch', 'trunc_normal_']


In [28]:
from MedVit_adapter import MedViT_small as small

In [29]:
!pip install medmnist

In [30]:
import medmnist
from medmnist import INFO , Evaluator
data_flag = 'retinamnist'
# [tissuemnist , pathmnist, chestmnist, dermamnist, octmnisr]
# ,pnemonismnist , retinamnist, bloodmnist, tissuemnist, organcmist, organs ]
download = True

NUM_EPOCHS = 15
BATCH_SIZE = 15
LR = 0.005

info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

DataClass = getattr(medmnist, info['python_class'])

In [31]:
from torchvision.transforms.transforms import Resize
import torchvision.transforms as transforms
#preprocessing
train_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.Lambda(lambda image:image.convert('RGB')),
    torchvision.transforms.AugMix(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
]
)
test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.Lambda(lambda image: image.convert('RGB')),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5],std=[0.5])
])

In [32]:
import torch.utils.data as data
# load the data
train_dataset = DataClass(split='train', transform=train_transform, download=download)
val_dataset = DataClass(split='val', transform=train_transform,download=download)
test_dataset = DataClass(split='test', transform=test_transform, download=download)


# encapsulate data into dataloader form
train_loader = data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
train_loader_at_eval = data.DataLoader(dataset=val_dataset, batch_size=2*BATCH_SIZE, shuffle=False)
test_loader = data.DataLoader(dataset=test_dataset, batch_size=2*BATCH_SIZE, shuffle=False)

Using downloaded and verified file: /root/.medmnist/retinamnist.npz
Using downloaded and verified file: /root/.medmnist/retinamnist.npz
Using downloaded and verified file: /root/.medmnist/retinamnist.npz


In [33]:
print(train_dataset)
print("+++++++++++++++++")
print(test_dataset)

Dataset RetinaMNIST (retinamnist)
    Number of datapoints: 1080
    Root location: /root/.medmnist
    Split: train
    Task: ordinal-regression
    Number of channels: 3
    Meaning of labels: {'0': '0', '1': '1', '2': '2', '3': '3', '4': '4'}
    Number of samples: {'train': 1080, 'val': 120, 'test': 400}
    Description: The RetinaMNIST is based on the DeepDRiD challenge, which provides a dataset of 1,600 retina fundus images. The task is ordinal regression for 5-level grading of diabetic retinopathy severity. We split the source training set with a ratio of 9:1 into training and validation set, and use the source validation set as the test set. The source images of 3×1,736×1,824 are center-cropped and resized into 3×28×28.
    License: CC BY 4.0
+++++++++++++++++
Dataset RetinaMNIST (retinamnist)
    Number of datapoints: 400
    Root location: /root/.medmnist
    Split: test
    Task: ordinal-regression
    Number of channels: 3
    Meaning of labels: {'0': '0', '1': '1', '2': '2

In [34]:
model = small()

initialize_weights...


In [35]:
model.proj_head[0] = nn.Linear(in_features=1024, out_features=n_classes, bias=True)

In [36]:
#Define loss function and optimizer
if task == "multi-task, binary-class":
  criterion = nn.BCEWithLogitsLoss()
else:
  criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9)

In [37]:
criterion

CrossEntropyLoss()

In [39]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [40]:
from torch.nn.functional import softmax
from tqdm import tqdm
# training
for epoch in range(NUM_EPOCHS):

  model = model.to(device)
  model.train()

  for inputs, targets in tqdm(train_loader):
    inputs, targets = inputs.to(device), targets.to(device)
    optimizer.zero_grad()
    outputs = model(inputs.to(torch.float32))

    if task == 'multi-label, binary-class':
      targets = targets.to(torch.float32).unsqueeze(1)
    else:
      targets = targets.to(torch.long)
      targets = targets.view(-1)


    predicted_classes = torch.argmax(outputs, dim=1)
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()

    # Delete tensors to free up memory
    del inputs, targets, outputs, predicted_classes
    # Empty the cache to clear up some more memory
    torch.cuda.empty_cache()


100%|██████████| 72/72 [00:55<00:00,  1.29it/s]


In [49]:
from sklearn.metrics import roc_auc_score, accuracy_score

# switch to evaluation mode
def test(split):
  model.eval()
  # Lists to store actual and predicted values
  actuals = []
  probas = []
  predictions = []

  if split == 'val':
        data_loader = train_loader_at_eval
  else:
        data_loader = test_loader
        
  with torch.no_grad():
      for inputs, targets in tqdm(data_loader):

          inputs, targets = inputs.to(device), targets.to(device)

          if task == 'multi-label, binary-class':
              targets = targets.to(torch.float32).unsqueeze(1)
          else:
              targets = targets.to(torch.long)
              targets = targets.view(-1)

          outputs = model(inputs.to(torch.float32))
          softmax_outputs = softmax(outputs, dim=1)

          _, predicted_classes = torch.max(outputs, 1)

          # Store the actual targets and predicted probabilities
          actuals.extend(targets.cpu().numpy())
          probas.extend(softmax_outputs.detach().cpu().numpy())
          # Probability of positive class
          predictions.extend(predicted_classes.cpu().numpy())
  print('\n')
  auc = roc_auc_score(actuals, probas, multi_class='ovr')
  accuracy = accuracy_score(actuals, predictions)

  print('AUC of the model:', auc)
  print('Accuracy of the model:', accuracy)

print('==> Evaluating...')
test('val')
test('test')


==> Evaluating...
train_loader_at_eval


100%|██████████| 4/4 [00:02<00:00,  1.52it/s]




AUC of the model: 0.7488379396375583
Accuracy of the model: 0.525
test_loader


100%|██████████| 14/14 [00:05<00:00,  2.76it/s]



AUC of the model: 0.6944425412322447
Accuracy of the model: 0.5025
